# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])
df_rj['uf'].value_counts()

uf
RJ    263
Name: count, dtype: int64

In [4]:
# 2) Seu código aqui
status = df_rj.status.value_counts()
print('Os três status das solicitações mais frequentes são:\n')

for i in range(3):
   print(f'{i+1}º {status.index[i]}: {status[i]}')

Os três status das solicitações mais frequentes são:

1º Deferido: 106
2º Arquivado: 53
3º Regularizado: 26


C:\Users\alima\AppData\Local\Temp\ipykernel_50292\2035380022.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{i+1}º {status.index[i]}: {status[i]}')


In [6]:
# 3) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])

df_rj['ano'] = df_rj['data_status'].str.slice(6,10)
df_rj

df_rj['ano'].value_counts().sort_index(ascending=True)

ano
2002     4
2003     1
2004     1
2006     4
2007    29
2008    37
2009     9
2010    19
2011    10
2012    13
2013    15
2014    20
2015     8
2016     4
2017     5
2018     4
2019    16
2020    15
2021     5
2022    11
2023    28
2024     5
Name: count, dtype: int64

In [5]:
# 4) Seu código aqui

import pandas as pd
import requests

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])

df_rj['data_status'] = df_rj['data_status'].astype(str)

df_rj['ano'] = df_rj['data_status'].str.slice(6, 10)

print("DataFrame com a coluna 'ano' extraída:")
print(df_rj.head())

frequencia_anos = df_rj['ano'].value_counts().sort_index(ascending=True)

print("Frequência de cada ano:")
print(frequencia_anos)

DataFrame com a coluna 'ano' extraída:
   id_pleito tipo_interessado          interessado  cod_ibge  uf num_pvl  \
0       5475        Município          Nova Iguaçu   3303500  RJ    None   
1      10433        Município             Mesquita   3302858  RJ    None   
2      10441        Município              Niterói   3303302  RJ    None   
3      10108        Município  São Pedro da Aldeia   3305208  RJ    None   
4       8590        Município         Belford Roxo   3300456  RJ    None   

                           status          num_processo        data_protocolo  \
0                       Arquivado  17944.000122/2010-29  2010-12-03T02:00:00Z   
1                       Arquivado  17944.001199/2008-00  2008-09-30T03:00:00Z   
2                        Deferido  17944.001200/2014-36  2014-09-08T03:00:00Z   
3                       Arquivado  17944.001122/2008-21  2009-03-06T03:00:00Z   
4  Arquivado por decurso de prazo  17944.000804/2011-12  2014-10-20T02:00:00Z   

                 

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [9]:
#1) Seu código aqui

def consultar_dados(uf, tipo_interessado):
    # Montar a URL da API dinamicamente
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    
    try:
        # Realiza a requisição GET para obter os dados
        response = requests.get(url)
        
        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Converte os dados recebidos para formato JSON
            dados_json = response.json()
            
            # Inspeciona a estrutura dos dados JSON
            if 'items' in dados_json:
                # Extrai os itens da resposta JSON
                items = dados_json['items']
                
                # Cria um DataFrame a partir dos itens
                df = pd.DataFrame(items)
                
                return df
            else:
                print("A estrutura dos dados JSON não contém a chave 'items'.")
                return None
        else:
            print(f"Erro ao consultar os dados: {response.status_code}")
            return None
    except Exception as e:
        print(f"Ocorreu um erro ao consultar a API: {e}")
        return None

# Exemplo de uso da função
uf = "BA"  # Estado da Bahia
tipo_interessado = "Estado"  # Tipo de interessado
dados_df = consultar_dados(uf, tipo_interessado)

# Exibe os dados obtidos no formato de DataFrame
if dados_df is not None:
    print(dados_df.head())
    
    # Salva o DataFrame em um arquivo CSV
    arquivo_csv = "solicitacoes_ba_estado.csv"
    dados_df.to_csv(arquivo_csv, index=False)
    print(f"Dados salvos no arquivo {arquivo_csv}")
else:
    print("Não foi possível obter os dados.")


   id_pleito tipo_interessado interessado  cod_ibge  uf num_pvl  \
0      18363           Estado       Bahia        29  BA    None   
1      18476           Estado       Bahia        29  BA    None   
2      18369           Estado       Bahia        29  BA    None   
3      18503           Estado       Bahia        29  BA    None   
4      18550           Estado       Bahia        29  BA    None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  19407.000007/2005-08   
1  Encaminhado à PGFN com manifestação técnica fa...  19407.000030/2005-94   
2  Encaminhado à PGFN com manifestação técnica fa...  19407.000008/2006-25   
3                                           Deferido  19407.000037/2002-63   
4                                           Deferido  19407.000048/2006-77   

         data_protocolo                                      tipo_operacao  \
0  2005-06-23T03:00:00Z  Operação contratual exter

In [6]:
# 2) Seu código aqui

def consultar_dados(uf, tipo_interessado):
    # Montar a URL da API dinamicamente
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    
    try:
        # Realiza a requisição GET para obter os dados
        response = requests.get(url)
        
        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Converte os dados recebidos para formato JSON
            dados_json = response.json()
            
            # Inspeciona a estrutura dos dados JSON
            if 'items' in dados_json:
                # Extrai os itens da resposta JSON
                items = dados_json['items']
                
                # Cria um DataFrame a partir dos itens
                df = pd.DataFrame(items)
                
                # Imprimir as primeiras linhas do DataFrame para inspecionar as colunas
                print("Estrutura do DataFrame:")
                print(df.head())
                
                return df
            else:
                print("A estrutura dos dados JSON não contém a chave 'items'.")
                return None
        else:
            print(f"Erro ao consultar os dados: {response.status_code}")
            return None
    except Exception as e:
        print(f"Ocorreu um erro ao consultar a API: {e}")
        return None

# Exemplo de uso da função
uf = "MG"  # Estado de Minas Gerais
tipo_interessado = "Estado"  # Tipo de interessado
dados_df = consultar_dados(uf, tipo_interessado)

# Verifica se a consulta foi bem-sucedida
if dados_df is not None:
    # Exibe as colunas disponíveis no DataFrame
    print("Colunas disponíveis no DataFrame:")
    print(dados_df.columns)
    
    # Verifica se a coluna 'status' existe antes de filtrar
    if 'status' in dados_df.columns:
        # Filtrando os dados para o estado de Minas Gerais e status desejado
        solic_arquivadas_mg = dados_df[dados_df['status'] == 'Arquivado por decurso de prazo']
        
        # Contando o número de solicitações
        num_solic_arquivadas_mg = solic_arquivadas_mg.shape[0]
        
        print(f"O número de solicitações para Minas Gerais com status 'Arquivado por decurso de prazo' é: {num_solic_arquivadas_mg}")
    else:
        print("A coluna 'situacao' não está presente no DataFrame.")
else:
    print("Não foi possível obter os dados.")


Estrutura do DataFrame:
   id_pleito tipo_interessado   interessado  cod_ibge  uf num_pvl  \
0       8255           Estado  Minas Gerais        31  MG    None   
1       7419           Estado  Minas Gerais        31  MG    None   
2      13549           Estado  Minas Gerais        31  MG    None   
3       5582           Estado  Minas Gerais        31  MG    None   
4       7032           Estado  Minas Gerais        31  MG    None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.000742/2013-19   
1                                           Deferido  17944.000571/2012-39   
2                                           Deferido  19405.000023/2006-93   
3                                 Arquivado a pedido  17944.000148/2013-10   
4  Encaminhado à PGFN com manifestação técnica fa...  17944.000482/2012-92   

         data_protocolo                                      tipo_operacao  \
0  2014-02-24T

In [14]:
# 3) Seu código aqui

def consultar_dados(uf, tipo_interessado):
    # Montar a URL da API dinamicamente com os parâmetros fornecidos
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    
    try:
        # Realiza a requisição GET para obter os dados
        response = requests.get(url)
        
        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Converte os dados recebidos para formato JSON
            dados_json = response.json()
            
            # Inspeciona a estrutura dos dados JSON
            if 'items' in dados_json:
                # Extrai os itens da resposta JSON
                items = dados_json['items']
                
                # Cria um DataFrame a partir dos itens
                df = pd.DataFrame(items)
                
                # Imprimir as primeiras linhas do DataFrame para inspecionar as colunas
                print("Estrutura do DataFrame:")
                print(df.head())
                
                return df
            else:
                print("A estrutura dos dados JSON não contém a chave 'items'.")
                return None
        else:
            print(f"Erro ao consultar os dados: {response.status_code}")
            return None
    except Exception as e:
        print(f"Ocorreu um erro ao consultar a API: {e}")
        return None

# Exemplo de uso da função
uf = "BA"  # Estado da Bahia
tipo_interessado = "Município"  # Tipo de interessado
dados_df = consultar_dados(uf, tipo_interessado)

# Verifica se a consulta foi bem-sucedida
if dados_df is not None:
    # Exibe as colunas disponíveis no DataFrame
    print("Colunas disponíveis no DataFrame:")
    print(dados_df.columns)
    
    # Verifica se a coluna 'situacao' existe antes de filtrar
    if 'situacao' in dados_df.columns:
        # Filtrando os dados para solicitações deferidas
        deferidas_ba = dados_df[dados_df['situacao'] == 'Deferido']
        
        # Agrupando por município e contando as solicitações deferidas
        solicitacoes_por_municipio = deferidas_ba['interessado'].value_counts()
        
        # Encontrando o município com o maior número de solicitações deferidas
        municipio_com_mais_deferidas = solicitacoes_por_municipio.idxmax()
        max_solicitacoes = solicitacoes_por_municipio.max()
        
        print(f"O município da Bahia com mais solicitações deferidas é {municipio_com_mais_deferidas} com {max_solicitacoes} solicitações.")
    else:
        print("A coluna 'situacao' não está presente no DataFrame.")
else:
    print("Não foi possível obter os dados.")


Estrutura do DataFrame:
   id_pleito tipo_interessado              interessado  cod_ibge  uf  \
0      42907        Município                    Uibaí   2932408  BA   
1      32183        Município      São Félix do Coribe   2929057  BA   
2      29668        Município  Muquém de São Francisco   2922250  BA   
3      27766        Município                 Salvador   2927408  BA   
4      30327        Município         Feira de Santana   2910800  BA   

                num_pvl                                             status  \
0  PVL02.001218/2020-13                    Em retificação pelo interessado   
1  PVL02.001337/2019-32                                  Deferido (PVL-IF)   
2  PVL02.002186/2018-59                                  Deferido (PVL-IF)   
3  PVL02.000404/2018-11  Encaminhado à PGFN com manifestação técnica fa...   
4  PVL02.002643/2018-13                                           Deferido   

           num_processo        data_protocolo  \
0  17944.103098/2020-51  

In [4]:
# 4) Seu código aqui

import pandas as pd
import requests

def consultar_dados(uf, tipo_interessado):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    
    try:
        # Realiza a requisição GET para obter os dados
        response = requests.get(url)
        
        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Converte os dados recebidos para formato JSON
            dados_json = response.json()
            
            if 'items' in dados_json:
                items = dados_json['items']

                df = pd.DataFrame(items)

                print('Estrutura')
                print(df.head())

                return df
            else:
                print('A estrutura do JSON não contém a chave "items".')
                return None
        else:
            print(f'Erro ao consultar os dados: {response.status_code}')
            return None
    except Exception as e:
        print(f'Ocorreu um erro ao consultar a API: {e}')
        return None